In [1]:
# For Google Colab use
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd '/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder/Util'    
except ModuleNotFoundError:
    pass

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder/Util


In [2]:
# Install `face_recognition` if needed
try:
    import face_recognition
except ModuleNotFoundError:
    !pip install face_recognition  

In [3]:
# Imports
import os
import re
from heapq import nsmallest
from math import ceil
from time import time
from random import randint, seed
from pipeline import *

from PIL import Image

In [4]:
# CD into root dir for easy access to all dirs
%cd '..'
root_datasets_path = './Celeb-DF-v2'
real_video_dir_path = f'{root_datasets_path}/Celeb-videos/Celeb-real'
fake_video_dir_path = f'{root_datasets_path}/Celeb-videos/Celeb-synthesis'

/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder


In [5]:
# Create test set 6 male actors and females actors, respectively
test_males = [0,1,2,3,4,5]
test_females = [7,8,10,11,12,13]
test_actor_ids = ['id'+str(num)+'_' for num in test_males+test_females]
print(f'Actors in test set = {test_actor_ids}')

Actors in test set = ['id0_', 'id1_', 'id2_', 'id3_', 'id4_', 'id5_', 'id7_', 'id8_', 'id10_', 'id11_', 'id12_', 'id13_']


In [6]:
# Pick the class to pre-process `video_dir_path` = {real,fake}_video_dir_path
video_dir_path = fake_video_dir_path
class_name = video_dir_path.split('-')[-1]

rnd_dir = f'{root_datasets_path}/Celeb-rnd-30/Celeb-{class_name}'
rnd_test_dir = f'{root_datasets_path}/Celeb-rnd-30-test/Celeb-{class_name}'

In [7]:
# figures out which videos still need to be pre-processed 

df_type = 'rnd'
videos_done_in_rnd = [re.sub('_\d+.png', '.mp4', img) for img in os.listdir(f'{root_datasets_path}/Celeb-{df_type}-30/Celeb-{class_name}')]

In [8]:
videos_done_in_rnd_test = [re.sub('_\d+.png', '.mp4', img) for img in os.listdir(f'{root_datasets_path}/Celeb-{df_type}-30-test/Celeb-{class_name}')]

In [9]:
videos_to_do = set(os.listdir(video_dir_path)) - set(videos_done_in_rnd)

In [10]:
videos_to_do = videos_to_do - set(videos_done_in_rnd_test)

# Pick a number of frames (subsection) to create each image from
# `interval_in_frames = 1` = extract every frame
# `interval_in_frames = 60` = extract every 1 second chunk (30fps)
interval_in_frames = 30
print(f'Number of videos yet to be pre-processed = {len(videos_to_do)}')

Number of videos yet to be pre-processed = 857


In [11]:
for video_name in tqdm(videos_to_do):
    # Locate video and extract all frames
    video = os.path.join(video_dir_path, video_name)
    video_name = video_name.replace('.mp4', '')
    frames = get_every_frame(video, interval=1)
    
    # Use frames to find all face locations
    face_coordinates = get_face_locations(frames, GPU=True, batch_size=96)
    
    # Create imgs for train-validation set
    test_set = ''
    if any([1 if actor_id in video_name else 0 for actor_id in test_actor_ids]):
        # Create imgs for test set (id0,id1,id2,id3,id4,id5,id7,id8,id10,id11,id12,id13)
        test_set = '-test'

    print(f' {len(face_coordinates)} / {len(frames)} (faces / frames')

    save_faces(sub_average(frames, face_coordinates, interval=interval_in_frames), 
               f'{root_datasets_path}/Celeb-avg-30{test_set}/Celeb-{class_name}/{video_name}')
    save_faces(sub_difference(frames, face_coordinates, interval=interval_in_frames), 
               f'{root_datasets_path}/Celeb-diff-30{test_set}/Celeb-{class_name}/{video_name}')
    save_faces(sub_frames_every_interval(frames, face_coordinates, interval=interval_in_frames),
               f'{root_datasets_path}/Celeb-rnd-30{test_set}/Celeb-{class_name}/{video_name}')

  0%|          | 0/857 [00:00<?, ?it/s]

 410 / 410 (faces / frames


  0%|          | 1/857 [00:13<3:12:55, 13.52s/it]

 460 / 460 (faces / frames


  0%|          | 2/857 [00:22<2:52:39, 12.12s/it]

...

100%|█████████▉| 855/857 [1:57:10<00:20, 10.25s/it]

 349 / 349 (faces / frames


100%|█████████▉| 856/857 [1:57:17<00:09,  9.41s/it]

 434 / 434 (faces / frames


100%|██████████| 857/857 [1:57:26<00:00,  8.22s/it]
